Переносим данные из json в csv для удобства работы с ними.

In [ ]:
import json
import csv
import numpy as np

def load_json(file_path):
    """Load JSON data from a file."""
    with open(file_path, 'r') as file:
        data = json.load(file)
    return data

def parse_dict_to_row(data_dict):
    """Parse a dictionary and return a list of string values."""
    row = []
    for key, value in data_dict.items():
        if isinstance(value, dict):
            # Recursively parse nested dictionaries
            row.extend(parse_dict_to_row(value))
        elif isinstance(value, list):
            # Flatten lists into the row
            for item in value:
                if isinstance(item, dict):
                    row.extend(parse_dict_to_row(item))
                else:
                    row.append(str(item))
        else:
            row.append(str(value))
    return row

def save_to_csv(data, csv_file_path):
    """Save a 2D array to a CSV file."""
    with open(csv_file_path, 'w', newline='') as file:
        writer = csv.writer(file)
        writer.writerows(data)

def json_to_csv(json_file_path, csv_file_path):
    """Convert a JSON file to a CSV file."""
    # Load JSON data
    json_data = load_json(json_file_path)

    # Parse JSON data into a 2D array
    rows = [["bio", "likes"]]
    for item in json_data:
        if isinstance(item, dict):
            rows.append(parse_dict_to_row(item))
    # Save the 2D array to a CSV file
    save_to_csv(rows, csv_file_path)

# Example usage
json_file_path = '/content/sample_data/profiles.json'  # Replace with your JSON file path
csv_file_path = '/content/sample_data/profiles.csv'  # Replace with your desired CSV file path
json_to_csv(json_file_path, csv_file_path)

Считаем данные, разобьем слова (сплит по пробелу), удалим знаки препинания (для корректной работы модели), переведем все слова в нижний регистр.

In [ ]:
import pandas as pd
import re

df = pd.read_csv('/content/sample_data/profiles.csv')
raw_bios = list(df['bio'])
raw_likes = list(df['likes'])
new_raw_bios = []
for bio in raw_bios:
    new_bio = ""
    res = re.split(r'\W+', bio)
    for word in res:
        if (len(word) != 0):
            letter = word[0]
            i = 0
            while i != len(word):
                letter = letter.lower()
                new_bio += letter
                i += 1
                if i != len(word):
                    letter = word[i]
            new_bio += " "
    new_raw_bios.append(new_bio)

bio = list(map(lambda x:set(x.split()), new_raw_bios))
print(bio)

[{'see', 'or', 'snacks', 'into', 'and', 'together', 'sports', 'watch', 'i', 'team', 'there', 'binge', 'something', 'as', 'up', 'a', 'long', 's', 'really', 'happening', 'here', 'grab', 'netflix', 'to', 'pizza', 'just', 'what', 'not', 'hit', 'definitely', 'can', 'like', 'slice', 'for', 'want', 'but', 'if', 'you', 'your', 'involved', 'cheer', 'me'}, {'book', 'might', 'and', 'decent', 'impression', 'cozy', 'spirit', 'seeking', 'i', 'passionate', 'crime', 'places', 'also', 'of', 'in', 'a', 'cat', 'recommend', 'get', 'adventurous', 'just', 'exploring', 'love', 'bonus', 'can', 'about', 'partner', 'travel', 'good', 're', 'do', 'along', 'points', 'value', 'but', 'new', 'if', 'we', 'with', 'you', 'hiking', 'nights', 'podcast'}, {'tunes', 'and', 'on', 'listening', 'jazz', 'find', 'i', 'heart', 'someone', 'give', 'great', 'the', 'amateur', 'who', 'it', 'weekends', 'to', 'looking', 'live', 'experimenting', 'recipe', 'music', 'll', 'can', 'food', 'good', 'promise', 'for', 'recipes', 'simmering', 'ch

Составим словарь из всех слов в био, сделаем соответствие между словом и его индексом

In [ ]:
vocab = set()
for words in bio:
  for word in words:
    if len(word) > 0:
      vocab.add(word)

vocab = list(vocab)
word2index = {}
for i, word in enumerate(vocab):
    word2index[word] = i

Заменяем слова в био на индексы для лучшей работы модели, составляем итоговый входной датасет и датасет, состоящий из "оценок" био.


In [ ]:
input_dataset = []
for mes in bio:
    mes_indices = []
    for word in mes:
        try:
            mes_indices.append(word2index[word])
        except Exception:
            pass
    input_dataset.append(mes_indices)

target_dataset = []
for label in df['likes']:
    target_dataset.append(label / 10)
len(input_dataset)

979

Теперь приступим к построению двухслойной модели с использованием метода one-hot в виде суммирования соответствующих строк матрицы wights0_1 для получения layer1 (тк матрица слева состоит из нулей и единиц), тренируем модель

In [ ]:
import sys
np.random.seed(1)

def sigmoid(x):
    return 1/(1 + np.exp(-x))

alpha = 0.01
it = 150
h_s = 50

# рандомно инициализируем веса, которые соединяют 3 слоя



w01 = 0.2 * np.random.random((len(vocab), h_s)) - 0.1
w12 = 0.2 * np.random.random((h_s, 1)) - 0.1

NICE = 0
ALL = 0

for wtf in range(it):
  for i in range(170, 979):
    ALL += 1
    x, y = (input_dataset[i], target_dataset[i])

    l1 = sigmoid(np.sum(w01[x], axis = 0))
    l2 = sigmoid(np.dot(l1, w12))


    l2_d = l2 - y
    l1_d = l2_d.dot(w12.T)
    w01[x] -= l1_d * alpha
    w12 -= np.outer(l1, l2_d) * alpha

    if np.abs(l2 - y) < 0.1:
      NICE += 1

    if i % 10 == 0:
      sys.stdout.write(str(NICE/ALL) + "\n")

Streaming output truncated to the last 5000 lines.
0.8184364191393724
0.8184618400235218
0.818487253790265
0.8185126604425906
0.8185380599834856
0.8185634524159355
0.8185748464455109
0.81860022662731
0.8186255997090625
0.8186509656937471
0.8186763245843414
0.8186876948673854
0.8187130415332784
0.8187383811134562
0.8187637136108898
0.8187890390285483
0.8188143573693991
0.8188396686364081
0.8188649728325395
0.8188902699607559
0.818915560024018
0.8189408430252851
0.8189661189675149
0.8189913878536633
0.8190166496866844
0.819041904469531
0.8190671522051539
0.8190923928965027
0.8191176265465248
0.8191428531581664
0.8191680727343718
0.8191932852780838
0.8192184907922434
0.8192436892797904
0.8192688807436623
0.8192940651867954
0.8193192426121243
0.8193304821475836
0.8193556474865238
0.819380805815913
0.8194059571386796
0.8194311014577503
0.81945623877605
0.819481369096502
0.8195064924220283
0.8195316087555488
0.8195567180999819
0.8195818204582447
0.8196069158332522
0.8196320042279182
0.819657

Прогоняем нашу натренированную модель на тестовых данных


In [ ]:
ALL = 170
NICE = 0
for i in range(0, ALL):
  x = input_dataset[i]
  y = target_dataset[i]
  l1 = sigmoid(np.sum(w01[x],axis=0))
  l2 = sigmoid(np.dot(l1,w12))
  if np.abs(l2 - y) < 0.5:
    NICE += 1

print(NICE / ALL)

0.9470588235294117
